TP1 Introduction à [PyTorch](https://pytorch.org/)
==========================



In [1]:
import numpy as np
import torch
import matplotlib.pyplot as plt
np.random.seed(42)

# **1**. chargement et affichage des données

In [ ]:
# data loading
data = np.loadtxt('dataset.csv')
print('shape =', data.shape)
print('size  =', data.size)
print('dtype =', data.dtype)

In [ ]:
# data plot
plt.figure()
plt.scatter(data[:,0], data[:,1], s=1.0)
plt.title('Input data')

# **2**. Normalisation des données

Centrer et réduire les échantillons $\lbrace x_i, y_i \rbrace_{i=1\ldots N}$ en soustrayant la moyenne et en divisant par l'écart type calculés indépendamment pour $\mathcal{X}$ et pour $\mathcal{Y}$.

Utiliser 
[`np.mean`](https://numpy.org/doc/stable/reference/generated/numpy.mean.html) et 
[`np.std`](https://numpy.org/doc/stable/reference/generated/numpy.std.html) 
en faisant attention au paramètre `axis`.

In [5]:
# ...

In [ ]:
# normalized data plot
plt.scatter(data[:,0], data[:,1], s=1.0)
plt.title('Normalized data')

# **3**. Donné d'entraînement et données de test

Diviser les `data` en deux nouvelles variables `data_train` et `data_test` dont les tailles sont fixées par un ratio $70/30$.

La division des données doit être **aléatoire**. Utiliser la fonction [`np.random.permutation`](https://numpy.org/doc/2.0/reference/random/generated/numpy.random.permutation.html) pour générer une permutation aléatoire de $N$ indices entre $0$ et $N-1$.


In [7]:
ratio = 0.70

# ...
data_train = None # ...
data_test = None # ...

In [ ]:
# plot train and test data
plt.figure()
plt.scatter(data_train[:,0], data_train[:,1], label='train', s=1.0)
plt.scatter(data_test[:,0], data_test[:,1], label='test', s=1.0)
plt.legend()
plt.title('Train and test data')

# **4**. Création de DataSet et de DataLoader

Compléter la classe `DataSet` qui hérite de la classe [`torch.utils.data.Dataset`](https://pytorch.org/docs/stable/data.html#torch.utils.data.Dataset) qui propose une interface de type '_map_' (en opposition au [`torch.utils.data.IterableDataset`](https://pytorch.org/docs/stable/data.html#torch.utils.data.IterableDataset) qui propose une interface de type '_iterable_') :
- `self.data` correspond à `data_train` ou `data_test`, soit un tableau Numpy de taille $M\times 2$
- `DataSet.__len__` retourne le nombre d'éléments
- `DataSet.__getitem__` retourne une paire $\lbrace x_i, y_i\rbrace$ de 2 nombres flottants correspondants au $i$-ème exemple

In [9]:
class DataSet(torch.utils.data.Dataset):
    def __init__(self, data: np.ndarray) -> None:
        self.data = data
    def __len__(self) -> int:
        # ...
        pass
    def __getitem__(self, i: int):
        # ...
        pass

Créer les deux datasets `dataset_train` et `dataset_test` à partir des données `data_train` et `data_test`, puis créer deux [`torch.utils.data.DataLoader`](https://pytorch.org/docs/stable/data.html#torch.utils.data.DataLoader) avec 
- `batch_size` égal à `16` (taille de _'mini'-batch_)
- `shuffle` égal à `True` (les données sont aléatoirement réordonnées à chaque début d'époque)

In [10]:
# ...
dataset_train = None # ...
dataset_test = None # ...

batch_size = 16

dataloader_train = None # ...
dataloader_test = None # ...


Afficher les $3$ premiers batches du dataloader d'entraînement.

In [ ]:
plt.figure()

# ...

plt.title(f'3 first (random) training batches of size {batch_size}')


# **5**. Création du réseau de neurones

Compléter la classe `NeuralNet` qui hérite de la classe [`torch.nn.Module`](https://pytorch.org/docs/stable/generated/torch.nn.Module.html#module).

Dans le constructeur `__init__`, insérer des couches grâce à la fonction `append` de `torch.nn.Sequential`.
Chaque _layer_ est une combinaison d'une couche linéaire (`torch.nn.Linear`) et d'une fonction d'activation non-linéaire (`torch.nn.ReLU`), **sauf** la dernière couche qui ne contient qu'une couche linéaire.
- la première couche a une dimension `in_features` égale à `dim_input`, c'est-à-dire $1$ (la dimension de $\mathcal{X}$).
- la dernière couche a une dimension `out_features` égale à `dim_output`, c'est-à-dire $1$ (la dimension de $\mathcal{Y}$).
- les couches cachées ont des dimensions définies par la liste `dim_hidden`

Le fonction `forward` effectue une passe 'en avant' du réseau en l'appliquant aux données d'entrées `x`. Il suffit d'appeler l'opérateur `()` de `self.layers` pour que chaque couche soit appelée une à une. Dans ce TP, `x` est de taille `batch_size x 1`.

In [12]:

class NeuralNet(torch.nn.Module):
    def __init__(self, dim_input=1, dim_output=1, dim_hidden=[16]):
        super(NeuralNet, self).__init__()
        self.layers = torch.nn.Sequential()
        # 1st layer
        # ...

        # hidden layers
        # ...

        # last layer
        # ...
        

    def forward(self, x):
        # ...
        pass


Afficher le détail de chaque couche en parcourant les paramètres du modèles avec la fonction `named_parameters` de `torch.nn.Module`.

Sommer le nombre de poids de chaque couche et afficher le nombre total de poids. 

In [ ]:
dim_hidden = [16]
model = NeuralNet(dim_hidden=dim_hidden)
print(model)

n_weights = 0
# ...

print('Total number of weights =', n_weights)

# **6**. Préparation de l'entraînement

Instancier le critère d'optimisation `criterion`, en choisisant une fonction de _loss_ adaptée au problème de regression parmi [les loss disponibles dans Pytorch](https://pytorch.org/docs/stable/nn.html#loss-functions).

Instancier l'`optimizer`, qui est responsable de la descente de gradient et de la mise-à-jour des paramètres du modèle, en choisisant l'algorithme d'optimisation adapté parmi [les algorithmes disponibles dans PyTorch](https://pytorch.org/docs/stable/optim.html#algorithms).

L'`optimizer` doit pouvoir être initialisé avec les paramètres du modèle (cf fonction `torch.nn.Module.parameters`) et un pas d'apprentissage (_learning rate_) `lr = 0.1`. 

In [14]:
criterion = None # ...

lr = 0.1

optimizer = None # ...

# **7**. Entraînement

Compléter la boucle d'entraînement.

#### 1. extract (x,y)

- `x` et `y` doivent avoir une dimension de `batch_size x 1` (utiliser `torch.Tensor.unsqueeze` si besoins)
- le type doit être `torch.float` (utiliser `torch.Tensor.to` si besoins) 

#### 2. reset gradients to 0 using [`torch.optim.Optimizer.zero_grad`](https://pytorch.org/docs/stable/generated/torch.optim.Optimizer.zero_grad.html)

#### 3. perform prediction 

- simple appel à l'opérateur `()` du modèle (qui appelle `forward` du modèle)

#### 4. compute loss using the criterion, y_pred, and y (ground truth)

- simple appel à l'opérateur `()` du critère

#### 5. compute gradient using [`torch.Tensor.backward`](https://pytorch.org/docs/stable/generated/torch.Tensor.backward.html) on the result of the loss

- attention: le critère `criterion` retourne un `torch.Tensor` (de taille `1` soit juste une valeurs) qui permet de démarrer la rétropropagation des gradients
- utiliser [`torch.Tensor.item`](https://pytorch.org/docs/stable/generated/torch.Tensor.item.html) pour récupérer la valeur de la loss dans un `float`

#### 6. update weights using one step of gradient descent with [`torch.optim.Optimizer.step`](https://pytorch.org/docs/stable/generated/torch.optim.Optimizer.step.html)

#### 7. every `K` iterations

- perform one evaluation on the test dataloader
- store test loss and training loss averaged over the `K` last iterations

Le critère de la loss est utilisé ici, mais ni la rétropropagation des gradients, ni l'`optimizer` ne sont utilisés, et le `model` est en mode `eval`. Attention à bien le remettre en mode `train` après l'évaluation.

#### 8. print best score on eval dataset 


In [ ]:
model.train() # enable gradients computation (contrary to model.eval())

n_epochs = 50

all_loss_train = []
all_loss_eval = []

for epoch in range(n_epochs):
    print(f'epoch {epoch+1}/{n_epochs}')
    for iter,xy in enumerate(dataloader_train):
        
        # 1. extract (x,y)
        x = None # ...
        y = None # ...

        # 2. reset gradients to 0 using torch.optim.Optimizer.zero_grad
        # ...

        # 3. perform prediction 
        y_pred = None # ...

        # 4. compute loss using the criterion, y_pred, and y (ground truth)
        # ...

        # 5. compute gradient using torch.Tensor.backward on the result of the loss
        # ...

        # 6. update weights using one step of gradient descent with torch.optim.Optimizer.step
        # ...

        # 7. every K iterations
        #    - perform one evaluation on the test dataloader
        #    - store test loss and training loss averaged over the K last iterations
        if (iter+1) % 30 == 0:
            loss_value_train = 0.0
            loss_value_eval = 0.0
            # ...
            print(f'  iter {iter+1}/{len(dataloader_train)} train={loss_value_train:.4f} test={loss_value_eval:.4f}')
            
# 8. print best score on eval dataset 
min_loss = None # ...
print(f'Min loss = {min_loss:.6f}')

In [ ]:
# print train and test loss values
plt.figure()
plt.plot(all_loss_train[1:], label='train')
plt.plot(all_loss_eval[1:], label='test')
plt.legend()
plt.ylabel('losses')
plt.title(f'batch_size={batch_size}       n_epochs={n_epochs}       lr={lr}       model={dim_hidden}', fontsize = 8)

# **8**. Visualisation des résultats

Appliquer le modèle (réseau de neurones) à une plage de `256` valeurs ordonnées qui vont de `-1.75` à `+1.75` en utilisant `torch.linspace` pour générer ces valeurs.

In [17]:
model.eval()
# ...

x_visu = None # ...
y_visu = None # ...


Afficher le résultat du réseau sous la forme d'une courbe (`plt.plot`).

Pour convertir un tenseur PyTorch `torch.Tensor` en tableau Numpy `np.ndarray` : 
- détacher le tenseur du graph de calcul de PyTorch avec `torch.Tensor.detach`
- puis convertir le tenseur détaché avec `torch.Tensor.numpy`

In [ ]:
# plot of training and test data + visualization
plt.figure()
plt.scatter(data_train[:,0], data_train[:,1], s=1.0, label='train')
plt.scatter(data_test[:,0], data_test[:,1], s=1.0, label='test')
plt.plot(x_visu, y_visu, 'k', linewidth=0.75, label='model')
plt.legend()
plt.title(f'batch_size={batch_size}       n_epochs={n_epochs}       lr={lr}       model={dim_hidden}', fontsize = 8)

# **9**. Ajustement des hyperparamètres

Relancer l'entraînement et la visualisation des résultats en effectuant, étape-par-étape les ajustements suivants :

**Attention** : toujours relancer les cellules à partir de la création du réseau de neurones (sinon on fait de l'entraînement incrémental) !

1. Oscillation dans la loss $\Rightarrow$ diminuer le learning rate à `lr=0.01` 
2. Loss diminue toujours $\Rightarrow$ augmenter le nombre d'époques à `n_epochs = 100`
3. Peu d'amélioration $\Rightarrow$ augmenter la profondeur du réseau avec `dim_hidden = [16,16]`
4. Ou alors $\Rightarrow$ augmenter la largeur du réseau avec `dim_hidden = [128]`
5. Deep neural network $\Rightarrow$ `dim_hidden = [256,256,256,256]`